<a href="https://colab.research.google.com/github/JoeJiraWat/Ai-Builders/blob/main/Signal_Autotune_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyworld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyworld: filename=pyworld-0.3.5-cp311-cp311-linux_x86_64.whl size=924311 sha256=40e55d4b53f6caabc843f7231c69c0ed45d6be2591a8c68127258323fcdba207
  Stored in directory: /root/.cache/pip/wheels/26/f0/db/ebcd5cdfe5ad7d229917d3a8db6f18f0cf40f099bf878e294d
Successfully built pyworld


In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np
import collections
from google.colab import files # สำหรับ Colab file upload
import time
import glob
import pyworld # ใช้สำหรับ Analyze/Synthesize คุณภาพสูง
from scipy.stats import mode as scipy_mode # อาจจะไม่จำเป็นถ้าใช้ Counter.most_common
from scipy.signal import medfilt # สำหรับ Smoothing F0 (Optional)

ส่วนฟังก์ชัน load_audio, extract_pitch, apply_ai_enhancement,
calculate_pitch_accuracy_metrics, plot_pitch_comparison

1. โหลดไฟล์เสียง

In [ ]:
YOUR_DATASET_BASE_PATH = "/content/drive/MyDrive/Datasets_For_Ai_builders/"
CLEAN_VOICE_DIR = os.path.join(YOUR_DATASET_BASE_PATH, "CleanVoice")
RAW_VOICE_DIR_REFERENCE_ONLY = os.path.join(YOUR_DATASET_BASE_PATH, "Raw Voice (wav)") # Path อ้างอิง ไม่ได้ใช้โหลดตรงๆแล้ว
HIGH_NOTE_DIR = os.path.join(YOUR_DATASET_BASE_PATH, "HighNote/HighNote(Edited)/")
OUTPUT_DIR_HEURISTIC_V2 = "/content/drive/MyDrive/Datasets_For_Ai_builders/Model/" # เปลี่ยนชื่อ Output dir

In [ ]:
os.makedirs(YOUR_DATASET_BASE_PATH, exist_ok=True)
os.makedirs(CLEAN_VOICE_DIR, exist_ok=True)
os.makedirs(RAW_VOICE_DIR_REFERENCE_ONLY, exist_ok=True)
os.makedirs(HIGH_NOTE_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR_HEURISTIC_V2, exist_ok=True)

การสร้างไฟล์ Dummy ถ้ายังไม่มีไฟล์ใดๆ ในโฟลเดอร์ที่ระบุ

In [ ]:
if not glob.glob(os.path.join(CLEAN_VOICE_DIR, '*.wav')):
    print(f"สร้างไฟล์เสียง Clean Voice ตัวอย่างใน '{CLEAN_VOICE_DIR}'")
    sr_dummy = 22050
    y_c4 = librosa.tone(frequency=librosa.note_to_hz('C4'), sr=sr_dummy, duration=2)
    try: sf.write(os.path.join(CLEAN_VOICE_DIR, "clean_C4_example.wav"), y_c4, sr_dummy)
    except Exception as e: print(f"Dummy Clean error: {e}")

if not glob.glob(os.path.join(HIGH_NOTE_DIR, '*.wav')):
    print(f"สร้างไฟล์เสียง HighNote ตัวอย่างใน '{HIGH_NOTE_DIR}'")
    sr_dummy = 22050;
    y_a5 = librosa.tone(frequency=librosa.note_to_hz('A5'), sr=sr_dummy, duration=2)
    try: sf.write(os.path.join(HIGH_NOTE_DIR, "high_A5_example.wav"), y_a5, sr_dummy)
    except Exception as e: print(f"Dummy HighNote error: {e}")


LoadAudio

In [ ]:
def load_audio(file_path):
    """Technique: โหลดข้อมูลเสียงจากไฟล์ และทำให้เป็น C-contiguous float64"""
    try:
        y, sr = librosa.load(file_path, sr=None, mono=True, res_type='kaiser_best')
        print(f"  [ASP] โหลดไฟล์ '{os.path.basename(file_path)}' (SR: {sr} Hz, Duration: {len(y)/sr:.2f}s)")
        # ทำให้เป็น C-contiguous และ float64 เพื่อ pyworld
        y_processed = np.ascontiguousarray(y, dtype=np.float64)
        return y_processed, sr
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการโหลดไฟล์: {file_path} ({e})"); return None, None

สกัดpitch

In [ ]:
def extract_pitch_world(y, sr, frame_period=5.0):
    """Technique: Pitch Extraction / F0 Estimation (ใช้ pyworld.dio + stonemask)
       รับ y เป็น float64 และ C-contiguous array
    """
    try:
        # ตรวจสอบ dtype ของ y ที่เข้ามาในฟังก์ชันนี้
        # print(f"  [ASP Debug] dtype of y in extract_pitch_world: {y.dtype}, flags: {y.flags}")
        if not y.flags['C_CONTIGUOUS']:
            print("  [ASP Warning] Input array to extract_pitch_world is not C-contiguous. Forcing.")
            y = np.ascontiguousarray(y, dtype=np.float64) # Double check
        elif y.dtype != np.float64:
            print(f"  [ASP Warning] Input array to extract_pitch_world is {y.dtype}, not float64. Forcing.")
            y = y.astype(np.float64)


        _f0, _time_axis = pyworld.dio(y, sr, frame_period=frame_period)
        f0 = pyworld.stonemask(y, _f0, _time_axis, sr) # Refine F0
        hop_length = int(frame_period / 1000.0 * sr)
        return f0, _time_axis, hop_length
    except ValueError as ve: # ดักจับ ValueError โดยเฉพาะ
        print(f"เกิด ValueError ระหว่างสกัด Pitch ด้วย pyworld: {ve}")
        print(f"  ตรวจสอบ dtype และ contiguity ของ input array: dtype={y.dtype}, C_CONTIGUOUS={y.flags['C_CONTIGUOUS']}")
        return np.array([]), np.array([]), int(frame_period / 1000.0 * sr)
    except Exception as e:
        print(f"เกิดข้อผิดพลาดทั่วไประหว่างสกัด Pitch ด้วย pyworld: {e}")
        return np.array([]), np.array([]), int(frame_period / 1000.0 * sr)

In [ ]:
def load_audio(file_path):
    """Technique: โหลดข้อมูลเสียงจากไฟล์"""
    try:
        y, sr = librosa.load(file_path, sr=None, mono=True, res_type='kaiser_best')
        print(f"  [ASP] โหลดไฟล์ '{os.path.basename(file_path)}' (Sample Rate: {sr} Hz)")
        return y.astype(np.float32), sr
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการโหลดไฟล์: {file_path} ({e})")
        return None, None

In [ ]:
def extract_pitch_librosa(y, sr, hop_length=256):
    """Technique: Pitch Extraction / Fundamental Frequency Estimation (ใช้ librosa.pyin)"""
    try:
        print(f"  [ASP] กำลังสกัด Pitch (F0) ด้วย librosa.pyin (hop={hop_length})...")
        f0, voiced_flag, voiced_probs = librosa.pyin(y,
                                                     fmin=librosa.note_to_hz('C2'),
                                                     fmax=librosa.note_to_hz('C7'),
                                                     hop_length=hop_length)
        print(f"  [ASP] สกัด F0 สำเร็จ ({len(f0)} frames)")
        return f0, voiced_flag, hop_length
    except Exception as e:
        print(f"เกิดข้อผิดพลาดระหว่างการสกัด Pitch ด้วย Librosa: {e}")
        return np.array([]), np.array([]), hop_length

แปลง f0 เป็น note

In [ ]:
def f0_to_notes(f0_contour, get_cents=False):
    notes_output = []
    for f_val in f0_contour:
        if f_val is not None and f_val > 0 and not np.isnan(f_val):
            try:
                if get_cents:
                    midi_val = librosa.hz_to_midi(f_val)
                    note_name_full = librosa.midi_to_note(midi_val, octave=True, cents=True)
                    notes_output.append(note_name_full)
                else:
                    note_name_octave = librosa.hz_to_note(f_val, octave=True, cents=False)
                    notes_output.append(note_name_octave)
            except Exception: pass
    return notes_output


หาโน๊ดหลัก

In [ ]:
def get_dominant_note_from_f0_list(f0_values_list):
    all_notes = []
    for f0_contour in f0_values_list:
        notes_from_contour = f0_to_notes(f0_contour, get_cents=False)
        all_notes.extend(notes_from_contour)
    if not all_notes: return "N/A"
    note_counts = collections.Counter(all_notes)
    dominant_note_info = note_counts.most_common(1)
    result = dominant_note_info[0][0] if dominant_note_info else "N/A"
    print(f"  [ASP] โน้ตหลักจาก F0 list: {result}")
    return result


Model โปรไฟล์ จาก Datasets cleanVoices

In [ ]:
def build_clean_voice_pitch_profile(clean_voice_dir, frame_period=5.0, num_samples_to_analyze=10):
    print(f"\n[Heuristic Model Building] สร้างโปรไฟล์ Pitch จาก '{clean_voice_dir}'...")
    pitch_profile_temp = {}
    clean_files = sorted(glob.glob(os.path.join(clean_voice_dir, '*.wav')))
    if not clean_files:
        print(f"  [Heuristic Model Building] ไม่พบไฟล์ .wav ใน '{clean_voice_dir}'")
        return {}
    files_to_process = clean_files
    if len(clean_files) > num_samples_to_analyze:
        files_to_process = np.random.choice(clean_files, size=num_samples_to_analyze, replace=False)
    for filepath in files_to_process:
        print(f"  [Heuristic Model Building] วิเคราะห์ Clean: {os.path.basename(filepath)}")
        y, sr = load_audio(filepath)
        if y is None: continue
        f0, _, _ = extract_pitch_world(y, sr, frame_period=frame_period)
        for f_hz in f0:
            if f_hz > 0 and not np.isnan(f_hz):
                try:
                    note_name_octave = librosa.hz_to_note(f_hz, octave=True, cents=False)
                    if note_name_octave not in pitch_profile_temp: pitch_profile_temp[note_name_octave] = []
                    pitch_profile_temp[note_name_octave].append(f_hz)
                except Exception: pass
    final_pitch_profile = {}
    for note, f0_values in pitch_profile_temp.items():
        if f0_values and len(f0_values) > 1: final_pitch_profile[note] = np.nanmedian(f0_values)
    if not final_pitch_profile: print("  [Heuristic Model Building] ไม่สามารถสร้างโปรไฟล์ Pitch")
    else: print(f"[Heuristic Model Building] สร้างโปรไฟล์ Pitch {len(final_pitch_profile)} โน้ตสำเร็จ")
    return final_pitch_profile


วิเคราะห์ Dataset HighNote เพื่อหา Median F0

In [ ]:
def analyze_high_note_dataset(high_note_dir, frame_period=5.0, num_samples_to_analyze=10):
    print(f"\n[Dataset Analysis] วิเคราะห์เสียงสูงจาก '{high_note_dir}'...")
    all_high_f0s = []
    high_note_files = sorted(glob.glob(os.path.join(high_note_dir, '*.wav')))
    if not high_note_files:
        print(f"  [Dataset Analysis] ไม่พบไฟล์ .wav ใน '{high_note_dir}'"); return np.nan
    files_to_process = high_note_files
    if len(high_note_files) > num_samples_to_analyze:
        files_to_process = np.random.choice(high_note_files, size=num_samples_to_analyze, replace=False)
    for filepath in files_to_process:
        print(f"  [Dataset Analysis] วิเคราะห์ HighNote: {os.path.basename(filepath)}")
        y, sr = load_audio(filepath)
        if y is None: continue
        f0, _, _ = extract_pitch_world(y, sr, frame_period=frame_period)
        valid_f0s = f0[f0 > 0 & ~np.isnan(f0)]
        if len(valid_f0s) > 0: all_high_f0s.extend(valid_f0s)
    if not all_high_f0s: print("  [Dataset Analysis] ไม่สามารถสกัด F0 จาก HighNote"); return np.nan
    median_high_f0 = np.nanmedian(all_high_f0s)
    print(f"[Dataset Analysis] Median F0 เสียงสูง: {median_high_f0:.2f} Hz")
    return median_high_f0


ปรับปรุงเสียงด้วย Heuristic Model

In [ ]:
def apply_heuristic_pitch_correction_v2(
    y, sr, # y ที่เข้ามาในฟังก์ชันนี้ ควรจะเป็น float64 และ C-contiguous จาก load_audio แล้ว
    clean_pitch_profile,
    median_f0_from_high_notes=np.nan,
    user_target_note=None,
    pitch_correction_strength=0.7,
    correction_threshold_cents=50,
    high_note_activation_hz=440.0,
    frame_period=5.0
):
    print("\n[Heuristic Correction V2] กำลังประมวลผลการแก้ไข Pitch...")

    # --- ตรวจสอบและบังคับให้ y เป็น float64 และ C-contiguous อีกครั้งเพื่อความแน่นอน ---
    if not isinstance(y, np.ndarray): # ถ้า y ไม่ใช่ numpy array (ไม่น่าเกิดขึ้นถ้ามาจาก load_audio)
        print("  [Heuristic Correction V2 Warning] y is not a numpy array. Attempting conversion.")
        y = np.array(y) # ลองแปลง

    if y.dtype != np.float64:
        print(f"  [Heuristic Correction V2 Warning] dtype ของ y ที่รับมาคือ {y.dtype}, กำลังแปลงเป็น float64.")
        y = y.astype(np.float64)
    if not y.flags['C_CONTIGUOUS']:
        print("  [Heuristic Correction V2 Warning] y ที่รับมาไม่ใช่ C-contiguous, กำลังแปลง.")
        y = np.ascontiguousarray(y, dtype=np.float64)
    # --------------------------------------------------------------------------------

    f0_original, time_axis, _ = extract_pitch_world(y, sr, frame_period=frame_period)

    # สกัด SP และ AP จากเสียง Original โดยใช้ F0 Original
    # y ที่ส่งเข้า pyworld.cheaptrick และ d4c ต้องเป็น float64 และ C-contiguous
    try:
        sp_original = pyworld.cheaptrick(y, f0_original, time_axis, sr)
        ap_original = pyworld.d4c(y, f0_original, time_axis, sr)
    except ValueError as ve_sp_ap:
        print(f"  [Heuristic Correction V2 ERROR] เกิด ValueError ขณะสกัด SP/AP: {ve_sp_ap}")
        print(f"    ตรวจสอบ dtype/contiguity ของ y ที่ใช้สกัด SP/AP: {y.dtype}, C_CONTIGUOUS={y.flags['C_CONTIGUOUS']}")
        print("    คืนค่าเสียงเดิม")
        return y.astype(np.float32) # คืนค่า float32 ตามปกติ
    except Exception as e_sp_ap:
        print(f"  [Heuristic Correction V2 ERROR] เกิดข้อผิดพลาดทั่วไปขณะสกัด SP/AP: {e_sp_ap}")
        print("    คืนค่าเสียงเดิม")
        return y.astype(np.float32)


    if f0_original is None or len(f0_original) == 0 or not np.any(f0_original > 0):
        print("  [Heuristic Correction V2] ไม่สามารถสกัด F0 ต้นฉบับที่ Valid ได้, คืนค่าเสียงเดิม")
        return y.astype(np.float32) # คืนค่า float32 ตามปกติ

    f0_corrected = np.copy(f0_original)
    num_frames = len(f0_original)
    frames_corrected_count = 0

    # ... (ส่วนตรรกะการหา effective_target_f0_hz และการแก้ไข f0_corrected เหมือนเดิมทุกประการ) ...
    target_freq_user_hz = np.nan
    if user_target_note:
        try: target_freq_user_hz = librosa.note_to_hz(user_target_note)
        except Exception: print(f"  คำเตือน: ไม่สามารถแปลง target note '{user_target_note}'")
    for i in range(num_frames):
        current_f0 = f0_original[i]
        if current_f0 <= 0 or np.isnan(current_f0): continue
        effective_target_f0_hz = np.nan; target_source_info = "None"
        if not np.isnan(target_freq_user_hz):
            midi_current = librosa.hz_to_midi(current_f0); midi_target_base = librosa.hz_to_midi(target_freq_user_hz)
            octave_diff = round((midi_current - midi_target_base) / 12.0)
            effective_target_f0_hz = librosa.midi_to_hz(midi_target_base + octave_diff * 12)
            target_source_info = f"User ({user_target_note})"
        elif current_f0 >= high_note_activation_hz and not np.isnan(median_f0_from_high_notes):
            midi_current = librosa.hz_to_midi(current_f0); midi_high_ref_base = librosa.hz_to_midi(median_f0_from_high_notes)
            octave_diff_high = round((midi_current - midi_high_ref_base) / 12.0)
            effective_target_f0_hz = librosa.midi_to_hz(midi_high_ref_base + octave_diff_high * 12)
            target_source_info = "HighNote Ref"
        else:
            try:
                current_note_name_octave = librosa.hz_to_note(current_f0, octave=True, cents=False)
                target_f0_from_profile = clean_pitch_profile.get(current_note_name_octave)
                if target_f0_from_profile is not None and not np.isnan(target_f0_from_profile):
                    effective_target_f0_hz = target_f0_from_profile
                    target_source_info = f"CleanVoice Profile ({current_note_name_octave})"
                else:
                    midi_current = librosa.hz_to_midi(current_f0); quantized_midi = np.round(midi_current)
                    effective_target_f0_hz = librosa.midi_to_hz(quantized_midi)
                    target_source_info = "Chromatic Quantize"
            except Exception:
                midi_current = librosa.hz_to_midi(current_f0); quantized_midi = np.round(midi_current)
                effective_target_f0_hz = librosa.midi_to_hz(quantized_midi)
                target_source_info = "Chromatic Quantize (Fallback)"
        if effective_target_f0_hz > 0 and not np.isnan(effective_target_f0_hz):
            cents_diff = 1200 * np.log2(current_f0 / effective_target_f0_hz)
            current_strength = pitch_correction_strength
            if abs(cents_diff) > 700: current_strength *= 0.4
            elif abs(cents_diff) > 400: current_strength *= 0.6
            elif abs(cents_diff) > 200: current_strength *= 0.8
            if abs(cents_diff) > correction_threshold_cents:
                frames_corrected_count += 1
                cents_to_correct = cents_diff * current_strength
                corrected_pitch = current_f0 * (2**(-cents_to_correct / 1200.0))
                f0_corrected[i] = max(1.0, corrected_pitch)
    # ... (สิ้นสุดส่วนตรรกะการแก้ไข F0) ...

    if frames_corrected_count > 0:
        print(f"  [Heuristic Correction V2] ทำการปรับแก้ Pitch ไป {frames_corrected_count} เฟรม")
    else:
        print(f"  [Heuristic Correction V2] ไม่พบ Pitch ที่เหิน/หลงเกินเกณฑ์ที่ต้องแก้ไข")

    try: # Optional Smoothing
        voiced_mask = f0_corrected > 0
        if np.any(voiced_mask) and len(f0_corrected[voiced_mask]) >=3 :
            f0_corrected[voiced_mask] = medfilt(f0_corrected[voiced_mask], kernel_size=3)
            print("  [Heuristic Correction V2] ใช้ Median Filter กับ F0 ที่แก้ไขแล้ว")
    except Exception as e_smooth: print(f"  คำเตือน: Smoothing F0 error: {e_smooth}")


    f0_corrected[f0_corrected <= 0] = 0
    f0_corrected[np.isnan(f0_corrected)] = 0
    print("  [Heuristic Correction V2] กำลังสังเคราะห์เสียงใหม่ด้วย pyworld...")
    y_synthesized = pyworld.synthesize(f0_corrected.astype(np.float64), # F0 ต้องเป็น float64
                                        sp_original.astype(np.float64), # SP ต้องเป็น float64
                                        ap_original.astype(np.float64), # AP ต้องเป็น float64
                                        sr, frame_period)
    print("  [Heuristic Correction V2] สังเคราะห์เสียงเสร็จสิ้น")
    # คืนค่าเป็น float32 ซึ่งเป็นมาตรฐานสำหรับไฟล์เสียงส่วนใหญ่ และ soundfile จัดการได้ดี
    return y_synthesized.astype(np.float32)

 โหลดไฟล์เสียง .wav

In [ ]:
def upload_and_get_wav_path_colab(prompt_message="กรุณาอัปโหลดไฟล์เสียงร้อง .wav ที่ต้องการปรับปรุง:"):
    print(prompt_message)
    try:
        uploaded = files.upload(); time.sleep(1)
        if not uploaded: print("ไม่ได้เลือกไฟล์ใดๆ"); return None
        wav_files = {k: v for k, v in uploaded.items() if k.lower().endswith('.wav')}
        if len(wav_files) == 0: print("ข้อผิดพลาด: ไม่พบไฟล์ .wav"); return None
        elif len(wav_files) > 1: print("คำเตือน: พบไฟล์ .wav มากกว่า 1 ไฟล์, จะใช้ไฟล์แรก")
        filename = list(wav_files.keys())[0]; content = wav_files[filename]
        temp_save_path = os.path.join("/content", filename)
        with open(temp_save_path, 'wb') as f: f.write(content)
        print(f"ไฟล์ '{filename}' ถูกอัปโหลดและบันทึกไว้ที่: {temp_save_path}")
        if not os.path.exists(temp_save_path) or os.path.getsize(temp_save_path) == 0:
             print(f"คำเตือน: ไฟล์ '{temp_save_path}' ไม่ได้ถูกเขียนอย่างสมบูรณ์"); return None
        return temp_save_path
    except Exception as e: print(f"เกิดข้อผิดพลาดระหว่างการอัปโหลดไฟล์: {e}"); return None


In [ ]:
if __name__ == "__main__":
    # ... (ส่วนต้นของ main เหมือนเดิม: สร้าง dir, โหลด profile, upload ไฟล์) ...
    print("-----------------------------------------------------------")
    print("🎤 โปรแกรมปรับปรุงเสียงร้องด้วย Heuristic Model V2 (Dtype Fix Attempt) 🎶")
    print("-----------------------------------------------------------")

    FRAME_PERIOD_ANALYSIS = 5.0

    clean_voice_profile = build_clean_voice_pitch_profile(CLEAN_VOICE_DIR, frame_period=FRAME_PERIOD_ANALYSIS, num_samples_to_analyze=10)
    median_high_f0_ref = analyze_high_note_dataset(HIGH_NOTE_DIR, frame_period=FRAME_PERIOD_ANALYSIS, num_samples_to_analyze=10)

    if not clean_voice_profile:
        print("!!! คำเตือน: ไม่สามารถสร้างโปรไฟล์ Pitch จาก CleanVoice ได้ !!!")

    print(f"\nขั้นตอนที่ 1: อัปโหลดไฟล์เสียง Input (.wav) ที่ต้องการปรับปรุง")
    uploaded_audio_path = upload_and_get_wav_path_colab() # ฟังก์ชันนี้ควรมีอยู่

    if uploaded_audio_path and os.path.exists(uploaded_audio_path):
        y_original, sr_original = load_audio(uploaded_audio_path) # load_audio คืน float64
        if y_original is not None:
            print(f"\n--- กำลังประมวลผลไฟล์: {os.path.basename(uploaded_audio_path)} ---")
            # ... (ส่วนวิเคราะห์เสียงต้นฉบับเหมือนเดิม) ...
            print("\nขั้นตอนที่ 2: วิเคราะห์เสียงต้นฉบับ")
            original_f0, _, _ = extract_pitch_world(y_original, sr_original, frame_period=FRAME_PERIOD_ANALYSIS)
            if original_f0 is not None and len(original_f0) > 0 and np.any(original_f0 > 0):
                original_notes = f0_to_notes(original_f0)
                dominant_original_note = get_dominant_note_from_f0_list([original_f0])
                print(f"เสียงต้นฉบับ - โน้ตหลัก: {dominant_original_note}")

                # ... (ส่วนรับ target_note_input และ parameters อื่นๆ เหมือนเดิม) ...
                print("\nขั้นตอนที่ 3: ระบุโน้ตเป้าหมาย (ถ้าต้องการ)")
                target_note_input = input("ป้อนโน้ตเป้าหมายเดียว (เช่น C4) หรือกด Enter เพื่อข้าม: ").strip()
                if not target_note_input: target_note_input = None
                else:
                     try: librosa.note_to_hz(target_note_input)
                     except Exception: print(f"คำเตือน: รูปแบบโน้ต '{target_note_input}'"); target_note_input=None

                print("\nขั้นตอนที่ 4: ตั้งค่าพารามิเตอร์การแก้ไข")
                try: strength = float(input(f"  ความแรงแก้ไข Pitch (0.1-1.0, default: 0.7): ").strip() or "0.7"); strength = np.clip(strength, 0.1, 1.0)
                except ValueError: strength = 0.7
                try: threshold = float(input(f"  เกณฑ์แก้ไข (Cents, default: 50): ").strip() or "50"); threshold = max(10, threshold)
                except ValueError: threshold = 50
                try: high_note_thresh_hz_input = float(input(f"  ความถี่เริ่ม HighNote Ref (Hz, default: 440): ").strip() or "440")
                except ValueError: high_note_thresh_hz_input = 440

                # y_original ที่ส่งเข้า apply_heuristic_pitch_correction_v2 เป็น float64
                y_corrected = apply_heuristic_pitch_correction_v2(
                    y_original, sr_original, # y_original เป็น float64
                    clean_pitch_profile=clean_voice_profile,
                    median_f0_from_high_notes=median_high_f0_ref,
                    user_target_note=target_note_input,
                    pitch_correction_strength=strength,
                    correction_threshold_cents=threshold,
                    high_note_activation_hz=high_note_thresh_hz_input,
                    frame_period=FRAME_PERIOD_ANALYSIS
                )

                print("\nขั้นตอนที่ 5: วิเคราะห์เสียงที่แก้ไขแล้ว")
                # y_corrected ที่ได้จาก apply_heuristic_pitch_correction_v2 ควรเป็น float64
                # เพื่อให้ extract_pitch_world ทำงานได้ถูกต้อง
                corrected_f0, _, _ = extract_pitch_world(y_corrected, sr_original, frame_period=FRAME_PERIOD_ANALYSIS)
                # ... (ส่วนวิเคราะห์และบันทึกไฟล์ output เหมือนเดิม) ...
                if corrected_f0 is not None and len(corrected_f0) > 0 and np.any(corrected_f0 > 0):
                    corrected_notes = f0_to_notes(corrected_f0)
                    dominant_corrected_note = get_dominant_note_from_f0_list([corrected_f0])
                    print(f"เสียงที่แก้ไขแล้ว - โน้ตหลัก: {dominant_corrected_note}")
                else:
                    print("ไม่สามารถสกัด Pitch จากไฟล์เสียงที่แก้ไขแล้วได้")

                base_in, ext_in = os.path.splitext(os.path.basename(uploaded_audio_path))
                output_file_name = f"{base_in}_heuristic_v2_fixed{ext_in}"
                output_file_path = os.path.join(OUTPUT_DIR_HEURISTIC_V2, output_file_name)
                try:
                    # บันทึกเป็น float32 เพื่อความเข้ากันได้ทั่วไป
                    sf.write(output_file_path, y_corrected.astype(np.float32), sr_original)
                    print(f"\nบันทึกไฟล์เสียงที่แก้ไขแล้วไปที่ '{output_file_path}'")
                except Exception as e: print(f"เกิดข้อผิดพลาดในการบันทึกไฟล์: {e}")
            else:
                print("ไม่สามารถสกัด Pitch จากไฟล์เสียงต้นฉบับได้ หรือไม่มี voiced frames")
        else:
            print("ไม่สามารถโหลดไฟล์เสียงที่อัปโหลดได้")
    else:
        print("การอัปโหลดไฟล์เสียงไม่สำเร็จ หรือไม่ได้เลือกไฟล์")

    print("\n--- โปรแกรมสิ้นสุดการทำงาน ---")

-----------------------------------------------------------
🎤 โปรแกรมปรับปรุงเสียงร้องด้วย Heuristic Model V2 (Dtype Fix Attempt) 🎶
-----------------------------------------------------------

[Heuristic Model Building] สร้างโปรไฟล์ Pitch จาก '/content/drive/MyDrive/Datasets_For_Ai_builders/CleanVoice'...
  [Heuristic Model Building] วิเคราะห์ Clean: Ashes Remain - On My Own (Pseudo Video)_vocal.wav
  [ASP] โหลดไฟล์ 'Ashes Remain - On My Own (Pseudo Video)_vocal.wav' (Sample Rate: 44100 Hz)
  [ASP Warning] Input array to extract_pitch_world is float32, not float64. Forcing.
  [Heuristic Model Building] วิเคราะห์ Clean: -  Tilly Birds _ The Wall Song _vocal.wav
  [ASP] โหลดไฟล์ '-  Tilly Birds _ The Wall Song _vocal.wav' (Sample Rate: 44100 Hz)
  [ASP Warning] Input array to extract_pitch_world is float32, not float64. Forcing.
  [Heuristic Model Building] วิเคราะห์ Clean: - THE TOYS Ost Offcial Lyrics_vocal.wav
  [ASP] โหลดไฟล์ '- THE TOYS Ost Offcial Lyrics_vocal.wav' (Sample Rate: 44

Saving First.wav to First (4).wav
ไฟล์ 'First (4).wav' ถูกอัปโหลดและบันทึกไว้ที่: /content/First (4).wav
  [ASP] โหลดไฟล์ 'First (4).wav' (Sample Rate: 44100 Hz)

--- กำลังประมวลผลไฟล์: First (4).wav ---

ขั้นตอนที่ 2: วิเคราะห์เสียงต้นฉบับ
  [ASP Warning] Input array to extract_pitch_world is float32, not float64. Forcing.
  [ASP] โน้ตหลักจาก F0 list: E3
เสียงต้นฉบับ - โน้ตหลัก: E3

ขั้นตอนที่ 3: ระบุโน้ตเป้าหมาย (ถ้าต้องการ)
ป้อนโน้ตเป้าหมายเดียว (เช่น C4) หรือกด Enter เพื่อข้าม: C6

ขั้นตอนที่ 4: ตั้งค่าพารามิเตอร์การแก้ไข
  ความแรงแก้ไข Pitch (0.1-1.0, default: 0.7): 0.5
  เกณฑ์แก้ไข (Cents, default: 50): 30
  ความถี่เริ่ม HighNote Ref (Hz, default: 440): 700

[Heuristic Correction V2] กำลังประมวลผลการแก้ไข Pitch...
  [Heuristic Correction V2 Warning] dtype ของ y ที่รับมาคือ float32, กำลังแปลงเป็น float64.
  [Heuristic Correction V2] ทำการปรับแก้ Pitch ไป 9849 เฟรม
  [Heuristic Correction V2] ใช้ Median Filter กับ F0 ที่แก้ไขแล้ว
  [Heuristic Correction V2] กำลังสังเคราะห์เสียงใหม่